# Trabajo Práctico 1: Análisis Exploratorio de Datos #
## Cientes potenciales a tarjetas de crédito. ##
El [DATASET](https://www.kaggle.com/datasets/sajidhussain3/jobathon-may-2021-credit-card-lead-prediction?select=train.csv) contiene información de clientes de un banco denominado Happy Customer Bank. Es un banco privado de tamaño medio que se ocupa de todo tipo de productos bancarios, como cuentas de ahorro, cuentas corrientes, productos de inversión, productos de crédito, entre otras ofertas.  

En este caso, el Happy Customer Bank desea realizar ventas cruzadas de sus tarjetas de crédito a sus clientes existentes. El banco ha identificado un conjunto de clientes que son elegibles para aceptar estas tarjetas de crédito.

El banco tiene la necesidad, de identificar a los clientes que podrían mostrar intención hacia una tarjeta de crédito recomendada.

<img src = "https://storage.googleapis.com/kaggle-datasets-images/1371711/2277422/2c35585d42b3747882e83e926d0f643e/dataset-cover.png?t=2021-05-27-18-40-42">


 --- 

## 1. Listado de variables y selección ###
### Variables de Entrada ##

- **ID:** Identificador único para cada fila.



- **Gender:** Género del cliente.



- **Age:** Edad del cliente (en años).


        
- **Region_Code:** Código de la Región para los clientes


        
- **Occupation:** Ocupación del cliente.

        
        
- **Channel_Code:** Código de Canal de Adquisición para el Cliente (Codificado)


        
- **Vintage:** Vendimia para el Cliente (En Meses). VER

        
        
- **Credit_Product:** 



- **Avg_Account_Balance:** Saldo de cuenta promedio. Saldo promedio de la cuenta del cliente en los últimos 12 meses

      
      
- **Is_Active:** Si el Cliente tiene activo algún producto de crédito. Si el cliente está activo en los últimos 3 meses. 
            0-> El cliente no esta activo | 1-> el cliente esta activo
            
            
--------

### Variable de Salida ###

- **Is_Lead:**  Si el Cliente está interesado en la Tarjeta de Crédito.
            0-> El cliente no esta interesado | 1-> el cliente esta interesado



In [1]:
import pandas as pd 
import pytz

In [2]:
passes = pd.read_csv('train.csv')

In [3]:
passes.sample(10)

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead
91813,XQWQNFJV,Female,48,RG283,Self_Employed,X3,104,No,555004,No,0
92258,KZOA7V76,Male,36,RG281,Self_Employed,X3,27,NaN,177866,Yes,1
42605,GENZSEVV,Female,49,RG284,Entrepreneur,X3,87,Yes,3792005,No,1
116558,TDX3GH3A,Male,42,RG272,Salaried,X2,93,NaN,597101,No,1
160414,6OLZ67TK,Male,47,RG283,Other,X2,91,No,1395795,Yes,0
241880,GAHD3NYV,Female,38,RG282,Self_Employed,X2,15,No,284319,Yes,0
21080,5Q9GRSFH,Female,55,RG254,Other,X3,109,No,1389290,Yes,0
157551,EN26HLXJ,Male,38,RG268,Self_Employed,X1,31,No,987722,No,0
219093,GXYJCVAO,Male,26,RG270,Other,X1,14,No,984037,No,0
217845,FLXMEGXZ,Female,28,RG268,Salaried,X1,31,No,1423768,No,0
